In [1]:
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model


def gen_adaptive(m, pcs, times, keep_thoughts=False, name="final"):
    xIpt, xOpt = map(lambda x: numpy.array(x, dtype='int8'), multi_training.getPieceSegment(pcs))
    all_outputs = [xOpt[0]]
    if keep_thoughts:
        all_thoughts = []
    m.start_slow_walk(xIpt[0])
    cons = 1
    for time in range(multi_training.batch_len * times):
        resdata = m.slow_walk_fun(cons)
        nnotes = numpy.sum(resdata[-1][:, 0])
        if nnotes < 2:
            if cons > 1:
                cons = 1
            cons -= 0.02
        else:
            cons += (1 - cons) * 0.3
        all_outputs.append(resdata[-1])
        if keep_thoughts:
            all_thoughts.append(resdata)
    noteStateMatrixToMidi(numpy.array(all_outputs), 'output/' + name)
    if keep_thoughts:
        pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))


def fetch_train_thoughts(m, pcs, batches, name="trainthoughts"):
    all_thoughts = []
    for i in range(batches):
        ipt, opt = multi_training.getPieceBatch(pcs)
        thoughts = m.update_thought_fun(ipt, opt)
        all_thoughts.append((ipt, opt, thoughts))
    pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))


pcs = multi_training.loadPieces("music")

m = model.Model([300, 300], [100, 50], dropout=0.5)

Using gpu device 0: GeForce GTX 860M (CNMeM is enabled with initial size: 80.0% of memory, CuDNN 4004)
model.py:367: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [3]:
batches = 500

multi_training.trainPiece(m, pcs, batches)

pickle.dump(m.learned_config, open("output/final_learned_config_{0}.p".format(batches), "wb"))

epoch 0, error=127650.9375
epoch 100, error=21160.375
epoch 200, error=14519.4873047
epoch 300, error=10987.1601562
epoch 400, error=10200.1669922


In [4]:
gen_adaptive(m,pcs,10,name="composition_{0}".format(batches))